In [ ]:
pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 7.6 MB/s eta 0:00:00


In [ ]:
import time
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

# Initialize Spotipy
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id='bf03985672a24kc18c53060d4d2be41a', client_secret='dac1146eeccbb64bb8a04562ac06ce6c'))

def getTrackIDs(playlist_id):
    track_ids = []
    offset = 0
    while True:
        response = sp.playlist_items(playlist_id, offset=offset, fields='items.track.id,total', additional_types=['track'])
        if not response['items']:
            break
        track_ids += [item['track']['id'] for item in response['items'] if item['track']]
        offset += len(response['items'])
    return track_ids

def getTrackDetails(track_id):
    track_info = sp.track(track_id)
    details = {
        'name': track_info['name'],
        'album': track_info['album']['name'],
        'artist': track_info['artists'][0]['name'],
        'release_date': track_info['album']['release_date'],
        'duration': track_info['duration_ms'],
        'popularity': track_info['popularity']
    }
    return details

def getAudioFeatures(track_ids):
    features_list = []
    for i in range(0, len(track_ids), 50): # Batching requests
        batch = track_ids[i:i+50]
        features = sp.audio_features(batch)
        features_list.extend(features)
        time.sleep(0.5) # Sleep to avoid rate limit
    return features_list

# Replace 'playlist_id' with your Spotify playlist ID
track_ids = getTrackIDs('7A0BB1t8whMe5CELdkOGC4')
track_details_list = []
audio_features_list = []

for track_id in track_ids:
    time.sleep(0.5)  # Increased sleep time
    track_details = getTrackDetails(track_id)
    track_details_list.append(track_details)

audio_features_list = getAudioFeatures(track_ids)

# Combining track details and audio features
combined_data = []
for details, features in zip(track_details_list, audio_features_list):
    combined_data.append({**details, **features})

tracks_df = pd.DataFrame(combined_data)
print(tracks_df.head())

                                            name  \
0                                Blinding Lights   
1                                   Shape of You   
2                              Someone You Loved   
3  Sunflower - Spider-Man: Into the Spider-Verse   
4                                   Dance Monkey   

                                               album         artist  \
0                                    Blinding Lights     The Weeknd   
1                                         ÷ (Deluxe)     Ed Sheeran   
2            Divinely Uninspired To A Hellish Extent  Lewis Capaldi   
3  Spider-Man: Into the Spider-Verse (Deluxe Edit...    Post Malone   
4        Dance Monkey (Stripped Back) / Dance Monkey    Tones And I   

  release_date  duration  popularity  danceability  energy  key  loudness  \
0   2019-11-29    201573          18         0.513   0.796    1    -4.075   
1   2017-03-03    233712          88         0.825   0.652    1    -3.183   
2   2019-05-17    182160     

In [ ]:
tracks_df.head()

,name,album,artist,release_date,duration,popularity,danceability,energy,key,loudness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Blinding Lights,Blinding Lights,The Weeknd,2019-11-29,201573,18,0.513,0.796,1,-4.075,...,0.0938,0.345,171.017,audio_features,0sf12qNH5qcw8qpgymFOqD,spotify:track:0sf12qNH5qcw8qpgymFOqD,https://api.spotify.com/v1/tracks/0sf12qNH5qcw...,https://api.spotify.com/v1/audio-analysis/0sf1...,201573,4
1,Shape of You,÷ (Deluxe),Ed Sheeran,2017-03-03,233712,88,0.825,0.652,1,-3.183,...,0.0931,0.931,95.977,audio_features,7qiZfU4dY1lWllzX7mPBI3,spotify:track:7qiZfU4dY1lWllzX7mPBI3,https://api.spotify.com/v1/tracks/7qiZfU4dY1lW...,https://api.spotify.com/v1/audio-analysis/7qiZ...,233713,4
2,Someone You Loved,Divinely Uninspired To A Hellish Extent,Lewis Capaldi,2019-05-17,182160,90,0.501,0.405,1,-5.679,...,0.1050,0.446,109.891,audio_features,7qEHsqek33rTcFNT9PFqLf,spotify:track:7qEHsqek33rTcFNT9PFqLf,https://api.spotify.com/v1/tracks/7qEHsqek33rT...,https://api.spotify.com/v1/audio-analysis/7qEH...,182161,4
3,Sunflower - Spider-Man: Into the Spider-Verse,Spider-Man: Into the Spider-Verse (Deluxe Edit...,Post Malone,2019-02-22,158040,56,0.761,0.479,2,-5.574,...,0.0704,0.912,89.913,audio_features,7dbka99KTWke5o9hRp0JoB,spotify:track:7dbka99KTWke5o9hRp0JoB,https://api.spotify.com/v1/tracks/7dbka99KTWke...,https://api.spotify.com/v1/audio-analysis/7dbk...,158040,4
4,Dance Monkey,Dance Monkey (Stripped Back) / Dance Monkey,Tones And I,2019-10-17,209438,78,0.824,0.588,6,-6.400,...,0.1490,0.513,98.027,audio_features,2XU0oxnq2qxCpomAAuJY8K,spotify:track:2XU0oxnq2qxCpomAAuJY8K,https://api.spotify.com/v1/tracks/2XU0oxnq2qxC...,https://api.spotify.com/v1/audio-analysis/2XU0...,209438,4


In [ ]:
import pandas as pd

# Assuming 'df' is your preprocessed DataFrame
tracks_df.to_csv('C:/Users/97450/OneDrive/Desktop/Second Semester/Machine Learning and Data Mining/clustering data/preprocessed_data.csv', index=False)

OSError: ignored